# Collaborative filtering

Source: https://itnext.io/what-are-the-top-recommendation-engine-algorithms-used-nowadays-646f588ce639

Collaborative filtering methids are built on a dataset of user/item feedbacks. In this notebook, we will be working on the **goodbooks-10k** dataset, which provides ratings given by users on books.

In [34]:
import numpy as np
import math

Dataset: **goodbooks-10k**

In [43]:
path_goodbooks = "/Users/stanislasbucaille/Desktop/data_bases/data_books/goodbooks-10k/"
ratings = np.loadtxt(path_goodbooks + 'ratings.csv', delimiter=',', skiprows=1)

In [36]:
np.random.shuffle(ratings)

In [37]:
ratings_int = []
for line in ratings:
    b, u, r = int(line[0]), int(line[1]), int(line[2])
    ratings_int.append([b, u, r])
ratings_int = np.array(ratings_int)

In [38]:
# Create dictionnaries
dic_ratings = {}
dic_users = {}
dic_books = {}

for line in ratings_int:
    b, u, r = line[0], line[1], line[2]
    
    # Ratings
    dic_ratings[(b, u)] = r
    
    # Users
    if u not in dic_users:
        dic_users[u] = [b]
    else:
        dic_users[u].append(b)
    
    # Books
    if b not in dic_books:
        dic_books[b] = [u]
    else:
        dic_books[b].append(u)

# Create lists
list_users = [u for u in dic_users]
list_books = [b for b in dic_books]

# Indexation
ids_users = {}
for i in range(len(list_users)):
    u = list_users[i]
    ids_users[u] = i
        
ids_books = {}
for i in range(len(list_books)):
    b = list_books[i]
    ids_books[b] = i

## 1. User-User

It recommends an item to a user if similar users liked this item before. The similarity between two users is computed from the amount of items they have in common in the dataset.

This algorithm is very efficient when the number of users is way smaller than the number of items.

In [6]:
def similarity_users(u1, u2):
    if u1 == u2:
        return 0
    else:
        s = 0
        for b in dic_users[u1]:
            if b in dic_users[u2]:
                s += 1
        return s

In [7]:
def knn_users(k, u):
    k_neighbors = []
    
    similarities = [similarity_users(u, v) for v in dic_users]
    ids = [v for v in dic_users]

    for i in range(k): 
        ind_v = np.argmax(similarities)
        v = ids[ind_v]
        k_neighbors.append(v)
        # Update similarities to make sure a point is not picked twice
        similarities[ind_v] = 0
    
    return k_neighbors

In [52]:
def prediction_users(k, u, b):
    u_neighbors = knn_users(k, u)
    
    r = 0
    for v in u_neighbors:
        try:
            r += dic_ratings[(b, v)]
        except:
            k -= 1
    
    return round(r/k)

In [53]:
u = 314
n = 100

error1 = 0
error2 = 0
for line in ratings_int[:n]:
    b, u, r = line[0], line[1], line[2]
    try: 
        r_pred = prediction_users(50, u, b)
        error1 += abs(r-r_pred)
        if r != r_pred:
            error2 += 1
    except:
        n -= 1

print(error1/n)        
print(error2/n)

0.8089887640449438
0.6629213483146067


## 2. Item-Item

It recommends items that are similar to the ones you previously liked. As before the similarity between two items is computed using the amount of users they have in common in the dataset.

This algorithm is best when the number of items is way smaller than the number of users

In [11]:
def similarity_books(b1, b2):
    if b1 == b2:
        return 0
    else:
        s = 0
        for u in dic_books[b1]:
            if u in dic_books[b2]:
                s += 1
        return s

In [12]:
def knn_books(k, b):
    k_neighbors = []
    
    similarities = [similarity_books(b, p) for p in dic_books]
    ids = [p for p in dic_books]

    for i in range(k): 
        ind_p = np.argmax(similarities)
        p = ids[ind_p]
        k_neighbors.append(p)
        # Update similarities to make sure a point is not picked twice
        similarities[ind_p] = 0
    
    return k_neighbors

In [54]:
def prediction_books(k, u, b):
    b_neighbors = knn_books(k, b)
    
    r = 0
    for p in b_neighbors:
        try:
            r += dic_ratings[(p, u)]
        except:
            k -= 1
    
    return round(r/k)

In [55]:
u = 314
n = 100

error1 = 0
error2 = 0
for line in ratings_int[:n]:
    b, u, r = line[0], line[1], line[2]
    try: 
        r_pred = prediction_books(50, u, b)
        error1 += abs(r-r_pred)
        if r != r_pred:
            error2 += 1
    except:
        n -= 1

print(error1/n)        
print(error2/n)

0.5952380952380952
0.44047619047619047


## 3. User-Item

It combines both approaches to generate recommendations. The simplest ones are based on **matrix factorization** techniques.

The factorization can be trained using:
- SVD: very computationally intensive
- ALS: for medium-scale datasets
- SGD: for large-scale datasetsbut, but will be very slow

In [52]:
n = len(list_users)
m = len(list_books)

print('n = ' + str(n))
print('m = ' + str(m))

n = 53424
m = 10000


### Latent Matrix Factorization

Source: https://surprise.readthedocs.io/en/stable/matrix_factorization.html

Lets note $P \in R ^{n x k}$ and $Q \in R ^{k x m}$

Optimization problem with regularization term: 
$$min_{L,R} L(P,Q)$$
We have: $$L(P,Q) = \sum_{(u,i)\in \Omega} (r_{ui}-\hat{r}_{ui})^2 + \lambda (||p_u||^2 + ||q_i||^2)$$

with, 
- $\hat{r}_{ui} = q_i^T p_u $
- $e_{ui} = r_{ui}-\hat{r}_{ui}$

and $\Omega$ the set of tuples where the rating already exists 

The Matrix Factorization model equation is:

$$\hat{r}_{ij} = \space u_{i}^T.v_j = \sum_{k = 0}^K u_{ik} v_{jk}$$ 

where, <v_u,v_i>
- $\omega_0$ : model bias
- $\omega_u$ : user bias
- $\omega_i$ : item bias
- $<v_u,v_i>$ : interaction between user $u$ and item $i$


$$ \text{Loss } = \sum_{(u_i,v_p, v_n)\space\in\space T} \max(\langle u_i,v_{p} \rangle \space -  \space u_i^T.v_{n}+ \space \alpha, 0) $$

where , $T$ is a set a triplets and
- $u_i^T.v_p$ : user to item+ interaction
- $u_i^T.v_n$ : user to item- interaction


$$ \hat{r}_{ij} = \langle u_i,v_j \rangle = \sum_{k=1}^K u_{ik}v_{jk}$$

$$ \text{Loss } = \sum_{(u_i,v_p, v_n)\space\in\space T} \max(\langle u_i,v_{p} \rangle \space -  \space \langle u_i,v_{n} \rangle + \space \alpha, 0) $$

$$ \hat{y}(x) \space := w_0 + \sum_{i=1}^n w_i x_i + \sum_{l=2}^d\sum_{i_1=1}^n ... \sum_{i_l=i_{l-1}+1}^n \left(\prod_{j=1}^l x_{i_j}\right) \left(\sum_{f=1}^{k_l}\prod_{j=1}^l  v_{i_j,f}^{(l)}\right) $$

$$ \hat{y}(x) \space = w_0 + \sum_{i=1}^n w_i x_i + \sum_{i=1}^n\sum_{j=i+1}^n \langle v_i,v_j \rangle x_i x_j $$

$$ \hat{y}(x) \space = w_0 + \sum_{i=1}^n w_i x_i + \sum_{c,d \space\in\space C} \sum_{\substack{i \space\in\space I_c \\ j \space\in\space I_d}} \langle v_i,v_j \rangle x_i x_j $$

$$ \hat{y}(x) \space = w_0 + \sum_{i=1}^n w_i x_i + \sum_{c,d \space\in\space C} \lambda_{cd} \sum_{\substack{i \space\in\space I_c \\ j \space\in\space I_d}} \langle v_i,v_j \rangle x_i x_j $$

$$(I_c)_{c \in C}$$
$$I = \{1, ..., n\}$$
$$\forall i \in \{1, ..., N\}, \space x^{(i)} = \left(x^{(i)}_1, ... , x^{(i)}_n\right)$$


To solve this optimization prpoblem we will use a **Gradient descent**.

We have:
- $p_u \leftarrow p_u + \gamma (e_{ui} . q_i - \lambda p_u)$
- $q_i \leftarrow q_i + \gamma (e_{ui} . p_u - \lambda q_i)$



In [53]:
gamma_ = 0.005 # learning rate
lambda_ = 0.02 # regularization term

In [54]:
def norme(V):
    return sum([v**2 for v in V.tolist()])

def loss(P, Q):
    s = 0
    for line in ratings_int:
        i, u, r_ui = ids_books[line[0]], ids_users[line[1]], line[2]
        e_ui = r_ui - np.dot(P[u],Q[i])
        s += e_ui ** 2 + lambda_ * (norme(P[u]) + norme(Q[i]))
    return s

In [55]:
def Gradient_descent(k, num_epochs):
    #randomly initialize user/item factors from a Gaussian
    P = np.random.normal(0,.1,(n, k))
    Q = np.random.normal(0,.1,(m, k))
    print(P.shape, Q.shape)

    for epoch in range(num_epochs):
        for line in ratings_int:
            i, u, r_ui = ids_books[line[0]], ids_users[line[1]], line[2]
            e_ui = r_ui - np.dot(P[u],Q[i])
            
            temp = P[u]
            P[u] +=  gamma_ * (e_ui * Q[i] - lambda_ * P[u])
            Q[i] +=  gamma_ * (e_ui * temp - lambda_ * Q[i])
        print(loss(P, Q))

    return P, Q

In [2]:
2.4/6

0.39999999999999997

In [56]:
P, Q = Gradient_descent(10, 100)

(53424, 10) (10000, 10)
15520731.327242896
15346211.260263296
13659460.476903863
9720601.04628521
5895851.791168863
3609742.9247699957
2414261.145094462
1779132.5237831143
1421231.8366552545
1206045.8159536244
1069214.676922158
978003.8309890237
914636.1152599276
868936.3220445919
834817.6833848476
808503.613030002
787580.9972283937
770469.7758721492
756112.7915451486
743788.5550889261
732995.3719922131
723378.1544347245
714681.3725723712
706718.3019353005
699350.579867223
692474.3762643394
686010.8732956144
679899.5993140377
674093.686603639
668556.4488083773
663258.879108588
658177.8011217883
653294.4893185084
648593.6316179949
644062.5442536498
639690.5745021672
635468.6445198099
631388.9019418821
627444.4517511417
623629.1503155715
619937.4471729242
616364.2635951524
612904.899542326
609554.9625564455
606310.3136117894
603167.026056598
600121.3546420601
597169.7122907954
594308.652775053
591534.8578709178
588845.1278710789
586236.3745830146
583705.6161371993
581249.9730824975
57886

In [57]:
u = 314
n = 100

error1 = 0
error2 = 0
for line in ratings_int[:n]:
    i, u, r = ids_books[line[0]], ids_users[line[1]], line[2]
    r_pred = round(np.dot(P[u],Q[i]))
    error1 += abs(r-r_pred)
    if r != r_pred:
        error2 += 1

print(error1/n)        
print(error2/n)

0.26
0.26


## Results

**error1** : average distance between the actual rating and the predicted one

**error2** : percentage of correctly predicted ratings

User-User method (k = 50):
- error1 = 0.8089887640449438
- error2 = 0.6629213483146067

Item-Item method (k = 50):
- error1 = 0.5952380952380952
- error2 = 0.44047619047619047

Latent Factorization method (gamma = 0.005, lambda = 0.02):
- error1 = 0.47
- error2 = 0.39

### Conclusion

We can see that the **User-User** method has some difficulties to predict the exact ratings, but is able to maintain an average distance between the actual rating and the predicted one smaller than 1. 

The **Item-Item** method performs better than the User-User method, giving a smaller error for both error1 and error2. It is able to predict correctly the exact rating more the half of the time. 

Finally, the **Latent Matrix Factorization** method gives the best performance of all. Its results for error1 and error2 are both under 0.5.

**Problem** : All these algorithms share the drawback that there is no efficient method to update the embeddings after adding a new item or a new user. This problem is called the **cold-start problem**.